In [44]:
import polars as pl
import numpy as np
import random
from datetime import datetime, timedelta

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [45]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df.lazy()

In [46]:
num_cars = 1000
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data
print(data.head())

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SLICE[offset: 0, len: 5]
  DF ["car_id", "timestamp", "latitude", "longitude"]; PROJECT */5 COLUMNS; SELECTION: None


In [47]:
data.collect()

car_id,timestamp,latitude,longitude,notification
str,str,f64,f64,str
"""car_0""","""2023-01-01T00:00:00""",17.682309,-108.399908,"""engine_check"""
"""car_0""","""2023-01-01T00:03:44.922217""",17.682309,-108.399908,null
"""car_0""","""2023-01-01T00:04:50.113028""",17.682309,-108.399908,"""tire_pressure"""
"""car_0""","""2023-01-01T00:04:08.891034""",17.682309,-108.399908,"""engine_check"""
"""car_0""","""2023-01-01T00:01:47.255901""",17.682309,-108.399908,"""low_fuel"""
…,…,…,…,…
"""car_999""","""2023-01-07T23:05:38.026463""",23.48918,-88.752422,"""engine_check"""
"""car_999""","""2023-01-07T23:02:42.248360""",23.48918,-88.752422,"""low_fuel"""
"""car_999""","""2023-01-07T23:31:52.109844""",23.495319,-88.758702,"""low_fuel"""


## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [48]:

#datos formato Y-M-D T H-M-S por eso %F T %T y el %.f es para los casos que tenga nanoseconds de la forma .X

data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime, "%FT%T%.f"))


### Ingenieria de variables

Dado que va a entrar a un modelo de machine learning es encesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt.
Rolling ve hacia atras
Dynamic divide dia en segmentos de 5 min

1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [49]:
def add_notification_time(df: pl.DataFrame):
    """
    Añade una columna 'notification_time' que contiene la diferencia de tiempo
    en minutos entre notificaciones consecutivas del mismo vehículo.
    
    Args:
        df: DataFrame de Polars con columnas 'car_id', 'timestamp'
        
    Returns:
        DataFrame con la nueva columna 'notification_time'
    """
    return (
        df.sort(["car_id", "timestamp"])#lo ordenamos por carro y tiempo
        .with_columns([
            pl.col("timestamp")# selecciona columna
            .diff()# calcula dif con anterior
            .over("car_id")#reinicia entre cada vehiculo
            .alias("notification_time")#crear y nombrar nueva columna
        ])
    )
    
data = add_notification_time(data)

In [50]:
data.collect()

car_id,timestamp,latitude,longitude,notification,notification_time
str,datetime[μs],f64,f64,str,duration[μs]
"""car_0""",2023-01-01 00:00:00,17.682309,-108.399908,"""engine_check""",null
"""car_0""",2023-01-01 00:01:47.255901,17.682309,-108.399908,"""low_fuel""",1m 47s 255901µs
"""car_0""",2023-01-01 00:03:44.922217,17.682309,-108.399908,null,1m 57s 666316µs
"""car_0""",2023-01-01 00:03:59.330581,17.682309,-108.399908,"""engine_check""",14s 408364µs
"""car_0""",2023-01-01 00:04:08.891034,17.682309,-108.399908,"""engine_check""",9s 560453µs
…,…,…,…,…,…
"""car_999""",2023-01-07 23:19:20.181986,23.48918,-88.752422,"""engine_check""",1m 57s 334601µs
"""car_999""",2023-01-07 23:21:03.154102,23.48918,-88.752422,null,1m 42s 972116µs
"""car_999""",2023-01-07 23:31:52.109844,23.495319,-88.758702,"""low_fuel""",10m 48s 955742µs


2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

In [51]:
def haversine_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """
    Calcula la distancia en kilómetros entre dos puntos usando la fórmula de Haversine.
    """
    R = 6371  # Radio de la Tierra en kilómetros
    
    # Convertir grados a radianes
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Diferencias en coordenadas
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c

def add_distance(df: pl.DataFrame) -> pl.DataFrame:
    """
    Añade una columna 'distance' que contiene la distancia recorrida en kilómetros
    desde la última notificación para cada vehículo.
    
    Args:
        df: DataFrame de Polars con columnas 'car_id', 'latitude', 'longitude'
        
    Returns:
        DataFrame con la nueva columna 'distance'
    """
    return (
        df.sort(["car_id", "timestamp"])
        .with_columns([
            pl.struct(["latitude", "longitude"])
            .shift()
            .over("car_id")
            .alias("prev_coords")
        ])
        .with_columns([
            pl.col("prev_coords").struct.field("latitude").alias("prev_lat"),
            pl.col("prev_coords").struct.field("longitude").alias("prev_lon")
        ])
        .with_columns([
            pl.struct(["latitude", "longitude", "prev_lat", "prev_lon"])
            .map_elements(lambda x: haversine_distance(
                x["latitude"], 
                x["longitude"], 
                x["prev_lat"] if x["prev_lat"] is not None else x["latitude"], 
                x["prev_lon"] if x["prev_lon"] is not None else x["longitude"]
            ))
            .alias("distance")
        ])
        .drop(["prev_coords", "prev_lat", "prev_lon"])
    )
data = add_distance(data)

In [52]:
data.collect()

/tmp/ipykernel_998/1239791909.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  data.collect()


car_id,timestamp,latitude,longitude,notification,notification_time,distance
str,datetime[μs],f64,f64,str,duration[μs],f64
"""car_0""",2023-01-01 00:00:00,17.682309,-108.399908,"""engine_check""",null,0.0
"""car_0""",2023-01-01 00:01:47.255901,17.682309,-108.399908,"""low_fuel""",1m 47s 255901µs,0.0
"""car_0""",2023-01-01 00:03:44.922217,17.682309,-108.399908,null,1m 57s 666316µs,0.0
"""car_0""",2023-01-01 00:03:59.330581,17.682309,-108.399908,"""engine_check""",14s 408364µs,0.0
"""car_0""",2023-01-01 00:04:08.891034,17.682309,-108.399908,"""engine_check""",9s 560453µs,0.0
…,…,…,…,…,…,…
"""car_999""",2023-01-07 23:19:20.181986,23.48918,-88.752422,"""engine_check""",1m 57s 334601µs,0.0
"""car_999""",2023-01-07 23:21:03.154102,23.48918,-88.752422,null,1m 42s 972116µs,0.0
"""car_999""",2023-01-07 23:31:52.109844,23.495319,-88.758702,"""low_fuel""",10m 48s 955742µs,0.93605


3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars especificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de `distance`


In [53]:
#Intervalos 5 minutos
def compute_interval_statistics(df: pl.DataFrame, interval_minutes: int = 5) -> pl.DataFrame:
    """
    Computa estadísticas por intervalos de tiempo para cada vehículo.
    
    Args:
        df: DataFrame de Polars con las columnas necesarias
        interval_minutes: Tamaño del intervalo en minutos
        
    Returns:
        DataFrame con las estadísticas por intervalo
    """
    return (
        df.group_by_dynamic(
            index_column="timestamp",
            by="car_id",
            every=f"{interval_minutes}m",
            closed="both"
        ).agg([
            # Estadísticas para notification_time (convertido a minutos varianza)
            pl.col("notification_time").mean().alias("notification_time_mean_minutes"),
            pl.col("notification_time").median().alias("notification_time_median_minutes"),
            pl.col("notification_time").dt.total_minutes().var().alias("notification_time_variance_minutes"),
            pl.col("notification_time").max().alias("notification_time_max_minutes"),
            pl.col("notification_time").min().alias("notification_time_min_minutes"),
            
            # Estadísticas para distance
            pl.col("distance").mean().alias("distance_mean"),
            pl.col("distance").median().alias("distance_median"),
            pl.col("distance").var().alias("distance_variance"),
            pl.col("distance").max().alias("distance_max"),
            pl.col("distance").min().alias("distance_min"),
            
            # Opcional: contar número de notificaciones en el intervalo
            pl.count().alias("notifications_count")
        ])
    )

data = compute_interval_statistics(data, interval_minutes=5)

/tmp/ipykernel_998/2560277083.py:14: DeprecationWarning: The argument `by` for `LazyFrame.group_by_dynamic` is deprecated. It has been renamed to `group_by`.
  df.group_by_dynamic(
/tmp/ipykernel_998/2560277083.py:35: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("notifications_count")


In [54]:
data.collect()

/tmp/ipykernel_998/1239791909.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  data.collect()


car_id,timestamp,notification_time_mean_minutes,notification_time_median_minutes,notification_time_variance_minutes,notification_time_max_minutes,notification_time_min_minutes,distance_mean,distance_median,distance_variance,distance_max,distance_min,notifications_count
str,datetime[μs],duration[μs],duration[μs],f64,duration[μs],duration[μs],f64,f64,f64,f64,f64,u32
"""car_0""",2023-01-01 00:00:00,48s 352171µs,21s 498167µs,0.266667,1m 57s 666316µs,9s 560453µs,0.0,0.0,0.0,0.0,0.0,7
"""car_0""",2023-01-01 00:05:00,1m 51s 640217µs,1m 51s 640217µs,0.5,2m 5s 560794µs,1m 37s 719641µs,0.0,0.0,0.0,0.0,0.0,2
"""car_0""",2023-01-01 00:10:00,51s 262751µs,56s 895936µs,0.2,1m 29s 305959µs,9s 242290µs,0.0,0.0,0.0,0.0,0.0,5
"""car_0""",2023-01-01 00:15:00,3m 15s 525299µs,3m 15s 525299µs,null,3m 15s 525299µs,3m 15s 525299µs,0.0,0.0,null,0.0,0.0,1
"""car_0""",2023-01-01 00:20:00,4m 17s 843348µs,4m 17s 843348µs,18.0,7m 4s 466551µs,1m 31s 220146µs,0.0,0.0,0.0,0.0,0.0,2
…,…,…,…,…,…,…,…,…,…,…,…,…
"""car_999""",2023-01-07 23:05:00,24s 17745µs,14s 337349µs,0.083333,1m 13s 706773µs,1s 862658µs,0.0,0.0,0.0,0.0,0.0,12
"""car_999""",2023-01-07 23:10:00,1m 54s 193213µs,1m 15s 706218µs,1.333333,3m 13s 348969µs,1m 13s 524454µs,0.0,0.0,0.0,0.0,0.0,3
"""car_999""",2023-01-07 23:15:00,1m 28s 568144µs,1m 57s 334601µs,1.0,2m 3s 249992µs,25s 119841µs,0.0,0.0,0.0,0.0,0.0,3
